In [1]:
import pandas as pd
import os
import pickle
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
cwd = os.getcwd()
dataPath1 = cwd + '\\data\\train.csv'
dataPath2 = cwd + '\\data\\test.csv'
df1 = pd.read_csv(dataPath1)
df2 = pd.read_csv(dataPath2)

In [3]:
# Remove rows with missing target and separate target from predictors
df1.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = df1.SalePrice
df1.drop(['SalePrice'], axis=1, inplace=True)

# Split into training and validation data sets
train_X, val_X, train_y, val_y = train_test_split(df1, y, shuffle=True, train_size=0.85, test_size=0.15, random_state=1)

In [4]:
# Select categorical columns with relatively low cardinality
cat_cols = [column for column in train_X.columns if train_X[column].nunique() < 10 and train_X[column].dtype == "object"]

# Select numerical columns
num_cols = [column for column in train_X.columns if train_X[column].dtype in ['int64', 'float64']]

# Keep selected columns only
work_cols = cat_cols + num_cols
X_train = train_X[work_cols].copy()
y_train = train_y.copy()
X_val = val_X[work_cols].copy()
y_val = val_y.copy()
X_test = df2[work_cols].copy()

In [5]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='mean')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

# Define model
model = XGBRegressor(n_estimators=500, learning_rate=0.05, n_jobs=4)

In [6]:
# Bundle preprocessing and modeling code in a pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Preprocessing of training data, fit model 
pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = pipeline.predict(X_val)

print('MAE: {:,.3f}'.format(mean_absolute_error(y_val, preds)))

MAE: 13,497.413


In [7]:
# Access the current working directory and output the model to a pickle file
cwd = os.getcwd()
modelPath = os.path.join(cwd,'model', 'xgb_housing_pipeline.pkl')
with open(modelPath, 'wb') as file:
	pickle.dump(model, file)

In [8]:
# Make predictions using the test data file
preds_test = pipeline.predict(X_test)
print(preds_test)

[119660.1  156944.48 184066.2  ... 156186.19 107592.   225163.28]


In [9]:
# Save predictions to a file
output = pd.DataFrame({'Id': df2.Id, 'SalePrice': preds_test})
output.to_csv('data\\output.csv', index=False)